# Caching VLLM

`LangChain` cung cấp lớp đệm (caching layer) tùy chọn cho LLM.

Điều này hữu ích vì hai lý do:

* Khi yêu cầu cùng một completion nhiều lần, nó có thể **giảm số lượng cuộc gọi API** đến nhà cung cấp LLM và do đó tiết kiệm chi phí.
* Bằng cách **giảm số lượng cuộc gọi API** đến nhà cung cấp LLM, nó có thể **cải thiện thời gian chạy của ứng dụng.**

Nhưng đôi khi bạn cần triển khai dịch vụ LLM của riêng mình, chẳng hạn như hệ thống tại chỗ (on-premise system) nơi bạn không thể truy cập các dịch vụ đám mây.
Trong hướng dẫn này, chúng ta sẽ sử dụng API tương thích OpenAI của `vllm` và sử dụng hai loại bộ đệm (cache), ```InMemoryCache``` và ```SQLiteCache```.
Ở cuối mỗi phần, chúng ta sẽ so sánh thời gian thực (wall times) trước và sau khi đệm.

Mặc dù đây là hướng dẫn dành cho trường hợp dịch vụ LLM cục bộ (local LLM service), chúng tôi sẽ nhắc bạn về cách sử dụng bộ đệm với dịch vụ API OpenAI trước tiên.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("Liệt kê 5 món ăn ngon ở {place}")
chain = prompt | llm | StrOutputParser()

answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và thơm.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng ngon và phổ biến ở Hà Nội, bánh mỳ pate thường được ăn kèm với pate, thịt nguội, rau sống và sốt.

4. Nem chua rán: Một món ăn vặt ngon và hấp dẫn của Hà Nội, nem chua rán là sự kết hợp giữa nem chua và bột chiên giòn, tạo nên hương vị độc đáo.

5. Bún ốc: Một món ăn truyền thống của Hà Nội, bún ốc gồm bún, ốc và nước dùng thơm ngon, tạo nên một món ăn ngon và lạ miệng.


In [6]:
%%time
answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và thơm.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate được làm từ bánh mỳ nướng giòn và pate thơm ngon.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, ngọt, cay và giòn rất hấp dẫn.

5. Chả cá Lã Vọng: Một món ăn đặc sản của Hà Nội, chả cá Lã Vọng gồm cá chẽm nướng trên bếp than, ăn kèm với bún, rau sống và nước mắm chua ngọt.
CPU times: user 12.7 ms, sys: 0 ns, total: 12.7 ms
Wall time: 3.82 s


## ```InMemoryCache```
First, cache the answer to the same question using `InMemoryCache`.

In [7]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

In [8]:
%%time
answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và ngon nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, thơm ngon và hấp dẫn.
2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc biệt và hấp dẫn.
3. Bánh mỳ pate: Một món ăn sáng ngon và phổ biến ở Hà Nội, bánh mỳ pate thường được ăn kèm với pate, thịt nguội, rau sống và sốt mayonnaise.
4. Nem chua rán: Một món ăn vặt ngon và độc đáo của Hà Nội, nem chua rán là sự kết hợp giữa nem chua và bột chiên giòn, tạo nên hương vị đặc biệt.
5. Bún bò Nam Bộ: Một món ăn ngon và phổ biến ở Hà Nội, bún bò Nam Bộ gồm bún, thịt bò nướng, rau sống và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.
CPU times: user 8.49 ms, sys: 2.85 ms, total: 11.3 ms
Wall time: 4.17 s


In [9]:
%%time
answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và ngon nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, thơm ngon và hấp dẫn.
2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc biệt và hấp dẫn.
3. Bánh mỳ pate: Một món ăn sáng ngon và phổ biến ở Hà Nội, bánh mỳ pate thường được ăn kèm với pate, thịt nguội, rau sống và sốt mayonnaise.
4. Nem chua rán: Một món ăn vặt ngon và độc đáo của Hà Nội, nem chua rán là sự kết hợp giữa nem chua và bột chiên giòn, tạo nên hương vị đặc biệt.
5. Bún bò Nam Bộ: Một món ăn ngon và phổ biến ở Hà Nội, bún bò Nam Bộ gồm bún, thịt bò nướng, rau sống và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.
CPU times: user 684 μs, sys: 68 μs, total: 752 μs
Wall time: 719 μs


> Note that if we set `InMemoryCache` again, the cache will be lost and the wall time will increase.

## ```SQLiteCache```
Now, we cache the answer to the same question by using `SQLiteCache`.

In [10]:
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [11]:
%%time
answer = chain.invoke("Vĩnh Phúc")
print(answer)

1. Lẩu cá rô đồng: Một món ăn truyền thống của vùng đất Vĩnh Phúc, lẩu cá rô đồng được chế biến từ cá rô đồng tươi ngon, thơm ngon và bổ dưỡng.
2. Gỏi cuốn Vĩnh Tường: Một món ăn ngon và lạ miệng của Vĩnh Phúc, gỏi cuốn Vĩnh Tường được làm từ các nguyên liệu tươi ngon như thịt heo, tôm, rau sống và bún.
3. Bún riêu Vĩnh Yên: Một món bún riêu ngon và đặc trưng của Vĩnh Phúc, bún riêu Vĩnh Yên có hương vị đậm đà, thơm ngon và hấp dẫn.
4. Cơm lam Vĩnh Tường: Một món ăn truyền thống của vùng đất Vĩnh Phúc, cơm lam Vĩnh Tường được chế biến từ gạo nếp, thịt heo, nấm và gia vị đặc trưng.
5. Bánh đa cua Sông Lô: Một món bánh ngon và độc đáo của Vĩnh Phúc, bánh đa cua Sông Lô được làm từ bột gạo, cua tươi và gia vị đặc trưng.
CPU times: user 22.2 ms, sys: 1.06 ms, total: 23.2 ms
Wall time: 4.62 s


In [12]:
%%time
answer = chain.invoke("Vĩnh Phúc")
print(answer)

1. Lẩu cá rô đồng: Một món ăn truyền thống của vùng đất Vĩnh Phúc, lẩu cá rô đồng được chế biến từ cá rô đồng tươi ngon, thơm ngon và bổ dưỡng.
2. Gỏi cuốn Vĩnh Tường: Một món ăn ngon và lạ miệng của Vĩnh Phúc, gỏi cuốn Vĩnh Tường được làm từ các nguyên liệu tươi ngon như thịt heo, tôm, rau sống và bún.
3. Bún riêu Vĩnh Yên: Một món bún riêu ngon và đặc trưng của Vĩnh Phúc, bún riêu Vĩnh Yên có hương vị đậm đà, thơm ngon và hấp dẫn.
4. Cơm lam Vĩnh Tường: Một món ăn truyền thống của vùng đất Vĩnh Phúc, cơm lam Vĩnh Tường được chế biến từ gạo nếp, thịt heo, nấm và gia vị đặc trưng.
5. Bánh đa cua Sông Lô: Một món bánh ngon và độc đáo của Vĩnh Phúc, bánh đa cua Sông Lô được làm từ bột gạo, cua tươi và gia vị đặc trưng.
CPU times: user 3.27 ms, sys: 135 μs, total: 3.4 ms
Wall time: 3.11 ms


> Note that if we use `SQLiteCache`, setting caching again does not delete stored cache.

In [13]:
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [14]:
%%time
answer = chain.invoke("Vĩnh Phúc")
print(answer)

1. Lẩu cá rô đồng: Một món ăn truyền thống của vùng đất Vĩnh Phúc, lẩu cá rô đồng được chế biến từ cá rô đồng tươi ngon, thơm ngon và bổ dưỡng.
2. Gỏi cuốn Vĩnh Tường: Một món ăn ngon và lạ miệng của Vĩnh Phúc, gỏi cuốn Vĩnh Tường được làm từ các nguyên liệu tươi ngon như thịt heo, tôm, rau sống và bún.
3. Bún riêu Vĩnh Yên: Một món bún riêu ngon và đặc trưng của Vĩnh Phúc, bún riêu Vĩnh Yên có hương vị đậm đà, thơm ngon và hấp dẫn.
4. Cơm lam Vĩnh Tường: Một món ăn truyền thống của vùng đất Vĩnh Phúc, cơm lam Vĩnh Tường được chế biến từ gạo nếp, thịt heo, nấm và gia vị đặc trưng.
5. Bánh đa cua Sông Lô: Một món bánh ngon và độc đáo của Vĩnh Phúc, bánh đa cua Sông Lô được làm từ bột gạo, cua tươi và gia vị đặc trưng.
CPU times: user 3.61 ms, sys: 0 ns, total: 3.61 ms
Wall time: 3.11 ms


## Setup Local LLM with ```VLLM```

```bash
vllm serve Qwen/Qwen2.5-0.5B-Instruct --dtype auto --api-key token-abc123
```

In [24]:
from langchain_community.llms import VLLMOpenAI

llm = VLLMOpenAI(
    model="Qwen/Qwen2.5-0.5B-Instruct", openai_api_key="token-abc123", openai_api_base="http://localhost:8000/v1"
)

In [25]:
chain = prompt | llm | StrOutputParser()

In [26]:
%%time
answer = chain.invoke("Bắc Ninh")
print(answer)

, Việt Nam
Đến với Việt Nam, mọi người sẽ thấy rằng nó là một đất nước văn minh, giàu kinh tế và có nhiều loại thức ăn ngon. Món ăn của các vùng miền khác nhau không chỉ đơn giản là một thứ gì đó mà còn thể hiện về cách sống và văn hóa dân gian của vùng đất đó. Có nhiều loại mì ngon đến từ Bắc Ninh, Việt Nam như:
1. Bánh Chảo: Đây chỉ là những chiếc bánh nhỏ, bánh tráng màu sắc mập mờ, đặc trưng của quê hương. Đáy bánh mềm, đường ngọt, vị đậm đà, thơm ngon. Bánh Chảo thường được làm từ dưa hấu, cơm xèo, nấm hương, bơ, rau củ tươi.
2. Bánh Cảm: Đây là loại bánh lớn, không chỉ mặn mà cũng rất hấp dẫn với vị chua ngọt đặc trưng. Trái cây, cơm, nấm hương đều được hòa tan vào bột bánh, tạo nên hương vị đặc trưng.
3. Bánh Cháo: Bánh này mướt, mềm mại, dai dẳng và rất phù hợp để ăn
CPU times: user 18.5 ms, sys: 1.55 ms, total: 20.1 ms
Wall time: 2min 16s


In [27]:
%%time
answer = chain.invoke("Bắc Ninh")
print(answer)

, Việt Nam
Đến với Việt Nam, mọi người sẽ thấy rằng nó là một đất nước văn minh, giàu kinh tế và có nhiều loại thức ăn ngon. Món ăn của các vùng miền khác nhau không chỉ đơn giản là một thứ gì đó mà còn thể hiện về cách sống và văn hóa dân gian của vùng đất đó. Có nhiều loại mì ngon đến từ Bắc Ninh, Việt Nam như:
1. Bánh Chảo: Đây chỉ là những chiếc bánh nhỏ, bánh tráng màu sắc mập mờ, đặc trưng của quê hương. Đáy bánh mềm, đường ngọt, vị đậm đà, thơm ngon. Bánh Chảo thường được làm từ dưa hấu, cơm xèo, nấm hương, bơ, rau củ tươi.
2. Bánh Cảm: Đây là loại bánh lớn, không chỉ mặn mà cũng rất hấp dẫn với vị chua ngọt đặc trưng. Trái cây, cơm, nấm hương đều được hòa tan vào bột bánh, tạo nên hương vị đặc trưng.
3. Bánh Cháo: Bánh này mướt, mềm mại, dai dẳng và rất phù hợp để ăn
CPU times: user 2.61 ms, sys: 222 μs, total: 2.84 ms
Wall time: 2.5 ms
